In [2]:
import datasets

ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")
documents = [doc["text"] for doc in ds]

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [3]:
print(f"Loaded {len(documents)} documents.")

Loaded 2647 documents.


In [4]:
def chunk_examples(examples):
    chunks = []
    for text in examples['text']:  # replace 'text' with your dataset's text field name
        # Split text into 200-character chunks
        chunks += [text[i:i + 200] for i in range(0, len(text), 200)]
    return {'chunks': chunks}


In [5]:
chunked_ds = ds.map(chunk_examples, batched=True, remove_columns=ds.column_names)

Map:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [6]:
print(chunked_ds[:10])

{'chunks': [' Create an Endpoint\n\nAfter your first login, you will be directed to the [Endpoint creation page](https://ui.endpoints.huggingface.co/new). As an example, this guide will go through the steps to deplo', 'y [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) for text classification. \n\n## 1. Enter the Hugging Face Repository ID and yo', 'ur desired endpoint name:\n\n<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_repository.png" alt="select repository" />\n\n## 2. Select your Cloud Provider', ' and region. Initially, only AWS will be available as a Cloud Provider with the `us-east-1` and `eu-west-1` regions. We will add Azure soon, and if you need to test Endpoints with other Cloud Provider', 's or regions, please let us know.\n\n<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_region.png" alt="select regio

In [7]:
print(ds.column_names)

['text', 'source']


In [8]:
texts = chunked_ds['chunks']

In [10]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight and effective

In [11]:
embeddings = embedder.encode(texts, batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/3383 [00:00<?, ?it/s]

In [12]:
embeddings = embedder.encode(texts, batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/3383 [00:00<?, ?it/s]

In [13]:
import faiss
import numpy as np

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 distance metric

index.add(np.array(embeddings))
print(f"Indexed {index.ntotal} vectors.")

Indexed 108237 vectors.


In [14]:
faiss.write_index(index, "rag_index.faiss")

import pickle
with open("rag_chunks.pkl", "wb") as f:
    pickle.dump(texts, f)

In [15]:
query = "Tell me about cat agility."

query_embedding = embedder.encode([query])

D, I = index.search(np.array(query_embedding), k=3)  # retrieve top 3 chunks

retrieved_chunks = [texts[i] for i in I[0]]

In [22]:
import ollama

context = "\n".join(retrieved_chunks)
prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"

messages = [{'role': 'user', 'content': prompt}]

response = ollama.chat(model='hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF:latest', messages=messages)

print(response['message']['content'])


Cat Agility Training!

Cat agility training, also known as feline agility or kitten agility, is a fun and challenging activity for cats. Here's the scoop:

**Benefits for cats:**

1. **Exercise**: Cat agility training provides mental and physical stimulation, helping to reduce stress and boredom.
2. **Socialization**: Interacting with other cats and toys can help socialize your cat, reducing aggression and fear-based behaviors.
3. **Playtime**: Agility training offers a fun way to engage in play with your cat, promoting exercise and bonding.

**Setting up a cat agility course:**

1. **Cat tower or platform**: Choose a sturdy cat tower or platform that allows your cat to climb and navigate.
2. **Tunnels and tubes**: Set up tunnels and tubes for your cat to explore and crawl through.
3. **Feeder hurdles**: Place small hurdles or obstacles at different levels to encourage problem-solving and agility.
4. **Cat toys and chews**: Add cat toys, such as feather wands or laser pointers, and che